In [16]:
import pandas as pd
import bs4 as bs
import urllib.request
import re
import nltk
import heapq
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.model_selection import train_test_split

In [2]:
# Web sayfasını çekmek ve analiz etmek
url = 'https://en.wikipedia.org/wiki/Turkey'
scraped_data = urllib.request.urlopen(url).read().decode('utf-8')  # UTF-8 ile okuma

In [3]:
# BeautifulSoup ile parse etme
parsed_article = bs.BeautifulSoup(scraped_data, 'lxml', from_encoding="utf-8")

c:\Users\MSI-NB\AppData\Local\Programs\Python\Python312\Lib\site-packages\bs4\__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [4]:
# Makale metnini çıkarma
paragraphs = parsed_article.find_all('p')
article_text = ""
for p in paragraphs:
    article_text += p.text

In [5]:
# Metni temizleme (Referanslar, özel karakterler vs.)
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)  # Referansları temizle
article_text = re.sub(r'\s+', ' ', article_text)  # Fazla boşlukları temizle
formatted_article_text = re.sub('[^a-zA-ZçğıöşüÇĞİÖŞÜ]', ' ', article_text)  # Türkçe harfleri koru
formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)  # Boşlukları normalize et

In [6]:
# Şimdi, metni pandas DataFrame'e dönüştürme
data = {
    'text': formatted_article_text.split('. ')  # Cümleleri bir liste olarak al
}

In [9]:
df = pd.DataFrame(data)
df

,text
0,Turkey a officially the Republic of Türkiye b...


In [10]:
df.head()

,text
0,Turkey a officially the Republic of Türkiye b...


In [8]:
df.columns

Index(['text'], dtype='object')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1 non-null      object
dtypes: object(1)
memory usage: 140.0+ bytes


In [12]:
df.describe()

,text
count,1
unique,1
top,Turkey a officially the Republic of Türkiye b...
freq,1


In [13]:
# Kelime sayısını hesaplayarak yeni bir sütun ekleyelim
df['word_count'] = df['text'].apply(lambda x: len(x.split()))

print(df.head())

                                                text  word_count
0   Turkey a officially the Republic of Türkiye b...       10479


In [14]:
# NLTK stop words
stop_words = set(stopwords.words('english'))  # İngilizce stop words

# Cümleleri ayırma
sentence_list = re.split(r'(?<=[.!?]) +', formatted_article_text)

# Kelime sıklığını hesaplama
word_frequencies = {}
for sentence in sentence_list:
    for word in sentence.split():
        word = word.lower()  # Küçük harfe çevir
        if word not in stop_words and word.isalpha():  # Sadece alfabe karakterlerini dikkate al
            word_frequencies[word] = word_frequencies.get(word, 0) + 1

In [22]:
# Etiketler (Örnek olarak 0 ve 1 etiketleri)
y = [1 if i % 2 == 0 else 0 for i in range(len(df))]  # Her 2. cümleye 1, diğerlerine 0

# X'i metin verisi olarak ayarlıyoruz
X = df['text']

# Eğitim ve test setlerine ayırma
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ayrılan eğitim ve test verilerini kontrol etme
print("X_train: ", X_train.head())
print("y_train: ", y_train[:5])
print("X_test: ", X_test.head())
print("y_test: ", y_test[:5])

ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

# Metin verisini vektörize etme (sayılara dönüştürme)
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Vektörize edilmiş veriyi kontrol etme
print("X_train_vec:\n", X_train_vec.toarray())

NameError: name 'X_train' is not defined